### This notebook contains the code for RAG implementation with Qdrant DB for Proposal Pro backend
Steps followed:
- Upload files
- for file in files:
    - for each page in file: 
        - update metadata
        - chunkify with metadata (RecursiveCharacterTextSplitter)
        - append to a docs_list
    - return docs_list
    - update metadata for each chunk in docs_list:
        - chunk.metadata.update({"tags": "", "flename": ""})
    - get embedding model to use
    - create/update vector DB with the docs_list, embedding model, using collection_name